In [ ]:
import numpy as np
import math
import json
import glob
from functools import reduce

In [ ]:
def load_results(dataset, model, metric, ref, n_sample, seed):
    result_dir = f'../results/{dataset}/{model}_{seed}/{metric}_{ref}_{n_sample}'
      
    result_fs = glob.glob(f'{result_dir}/*.json')
    results = []
    for file in result_fs:
        with open(file, 'r') as f:
            data = json.load(f)
            print(file, len(data))
        results.extend(data)
    
    if 'wd' in metric:
        for r in results:
            r['mismatch_rate'] = 1 - r['match_rate']

    return results

In [ ]:
metric = 'wd'
dataset = 'CIFAR-20'
arch = 'resnet50'
n_sample = 10000
ref = 'val'
seed = '1'
logits_results = load_results(dataset, arch, metric, ref, n_sample, seed)

In [ ]:
len(logits_results)

In [ ]:
import altair as alt
import pandas as pd

In [ ]:
corr = alt.Chart(alt.Data(values=logits_results)).mark_point().encode(
    x=alt.X('metric:Q', title=metric),
    y='error:Q',
    color=alt.Color('corruption:N', scale=alt.Scale(scheme='category20')),
    shape='corruption level:O'
).properties(
    width=250,
    height=250
)

if metric == 'wd':
    match = alt.Chart(alt.Data(values=logits_results)).mark_point().encode(
        x=alt.X('mismatch_rate:Q', title=metric),
        y='error:Q',
        color=alt.Color('corruption:N', scale=alt.Scale(scheme='category20')),
        shape='corruption level:O'
    ).properties(
        width=250,
        height=250
    )

    line = pd.DataFrame({
        'metric': [0, 1],
        'error': [0, 1],
        
    })

    line_plot = alt.Chart(line).mark_line(color='black', strokeDash=[5, 8]).encode(
        x='metric',
        y='error',
    )


corr | (match + line_plot) if metric == 'wd' else corr


In [ ]:
def polyfit(x, y, degree=1):
    results = {}

    coeffs = np.polyfit(x, y, degree)

    results['polynomial'] = coeffs.tolist()

    p = np.poly1d(coeffs)

    yhat = p(x)                
    ybar = np.sum(y)/len(y)          
    ssreg = np.sum((yhat - ybar)**2)   
    sstot = np.sum((y - ybar)**2)    
    results['determination'] = ssreg / sstot

    return results

In [ ]:
from scipy.stats import spearmanr
import math

In [ ]:
d = [i['metric'] for i in logits_results]
e = [i['error'] for i in logits_results]
print(f'{metric} r2:', polyfit(d, e)['determination'])

coef, p = spearmanr(d, e)
print(f'{metric} rho:', coef)

In [ ]:
print('slope:', polyfit(d, e)['polynomial'][0])
print('bias:', polyfit(d, e)['polynomial'][1])

In [ ]:
if metric == 'wd':
    yhat = np.array([i['mismatch_rate'] for i in logits_results])
else:
    yhat = np.array(d)

y = np.array(e)

print('MAE:', np.abs(yhat - y).mean() * 100)